In [1]:
import numpy as np
from scipy.optimize import fsolve

# Constants
R = 0.6
maturities = np.array([3, 5, 10])  # years
cds_spreads = np.array([50, 60, 100]) * 1e-4  # convert from bp to decimal

# Function to calculate CDS spread for a given hazard rate
def cds_spread(hazard_rate, T, R):
    return (1 - R) * (hazard_rate * (1 - np.exp(-hazard_rate * T))) / (1 - np.exp(-hazard_rate * T))

# Function to find hazard rate that gives the observed CDS spread
def find_hazard_rate(hazard_rate_guess, spread, T, R):
    return cds_spread(hazard_rate_guess, T, R) - spread

# Initial guess for hazard rates
hazard_rate_guess = np.array([0.01, 0.01, 0.01])

# Solve for hazard rates
hazard_rates = fsolve(find_hazard_rate, hazard_rate_guess, args=(cds_spreads, maturities, R))

# Print results
for T, spread, hazard_rate in zip(maturities, cds_spreads, hazard_rates):
    print(f'Maturity: {T} years, CDS Spread: {spread * 1e4} bp, Hazard Rate: {hazard_rate}')


Maturity: 3 years, CDS Spread: 50.0 bp, Hazard Rate: 0.0125
Maturity: 5 years, CDS Spread: 60.0 bp, Hazard Rate: 0.015
Maturity: 10 years, CDS Spread: 100.0 bp, Hazard Rate: 0.024999999999999994


In [2]:
from scipy.integrate import quad

# Bond parameters
coupon_rate = 0.02  # 2% coupon
coupon_frequency = 2  # semi-annual coupons
face_value = 100  # face value
bond_maturity = 6  # years

# Function to interpolate hazard rates
def interpolated_hazard_rate(t):
    if t <= 3:
        return hazard_rates[0]
    elif t <= 5:
        return hazard_rates[1]
    else:
        return hazard_rates[2]

# Function to integrate for bond pricing
def bond_price_integral(t):
    return coupon_rate / coupon_frequency * face_value * np.exp(-interpolated_hazard_rate(t) * t)

# Calculate present value of coupon payments
pv_coupons, _ = quad(bond_price_integral, 0, bond_maturity)

# Calculate present value of face value
pv_face_value = face_value * np.exp(-interpolated_hazard_rate(bond_maturity) * bond_maturity)

# Total bond price is sum of present value of coupons and face value
bond_price = pv_coupons + pv_face_value

print(f'The price of the bond is: {bond_price}')


The price of the bond is: 91.77040097152468
